In [234]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import albumentations as A
from pathlib import Path
import os
from tqdm import tqdm

In [235]:
#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

In [236]:
tf.__version__

'2.1.0'

In [237]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [326]:
train_df = pd.read_csv('../data/train.csv')
strs = train_df['image_id'].values
strs = [string+'_f.png' for string in strs]
train_df.drop(['image_id'],axis=1,inplace=True)
train_df['image_id'] = strs

In [327]:
train_df = train_df.astype(str)
train_df.head()

,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme,image_id
0,15,9,5,ক্ট্রো,Train_0_f.png
1,159,0,0,হ,Train_1_f.png
2,22,3,5,খ্রী,Train_2_f.png
3,53,2,2,র্টি,Train_3_f.png
4,71,9,5,থ্রো,Train_4_f.png


In [328]:
def apply_augmentation(image):
    augmentation_pipeline = A.Compose(
        [
            A.OneOf(
                [
                    # apply one of transforms to 50% of images
                    A.RandomContrast(), # apply random contrast
                    A.RandomGamma(), # apply random gamma
                    A.RandomBrightness(), # apply random brightness
                ],
                p = 0.5
            ),
            A.OneOf(
                [
                    # apply one of transforms to 50% images
                    A.ElasticTransform(alpha = 120,sigma = 120 * 0.05,alpha_affine = 120 * 0.03),
                    A.GridDistortion(),
                    A.OpticalDistortion(distort_limit = 2,shift_limit = 0.5),
                ],
                p = 0.5
            )
        ],
        p = 1
    )
    images_aug = augmentation_pipeline(image = image)['image']
    return images_aug

In [369]:
im_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                               preprocessing_function=apply_augmentation)


gen_image = im_generator.flow_from_dataframe(train_df,
                                             directory='/home/sayan/Documents/Bengali_Grapheme/images/processed_images',
                                             x_col='image_id',
                                             y_col=['grapheme_root','vowel_diacritic','consonant_diacritic'],
                                             target_size = (64,64),
                                             class_mode = 'multi_output',
                                             color_mode = 'grayscale',
                                             batch_size = 1,
                                             shuffle=False, 
                                             seed=42)

Found 200840 validated image filenames.
